In [ ]:
import pandas as pd
import numpy as np
import evaluation

## load_data

In [ ]:
# load prices
prices = pd.read_csv('close_prices_insample.csv', index_col='AsOfDate', parse_dates=['AsOfDate'])
ret = prices.diff()

In [ ]:
prices.plot()

## enter your own trading strategy here

## soft clipping

In [ ]:
def soft_clip(x, lower_bound=-1, upper_bound=1, smoothness=10):
    """
    Softly clip values using a tanh-based function to gradually bring extreme values closer
    to the target range.

    Parameters:
    - x: Input array or value
    - lower_bound: Lower bound for clipping
    - upper_bound: Upper bound for clipping
    - smoothness: The rate at which values are clipped; higher values make the transition smoother
    
    Returns:
    - Softly clipped values
    """
    # Scale the values to the range [0, 1] for tanh scaling
    scale = (x - lower_bound) / (upper_bound - lower_bound)
    # Apply tanh transformation and rescale back to the original range
    clipped = (np.tanh(smoothness * (scale - 0.5)) + 1) / 2  # range [0, 1]
    return lower_bound + clipped * (upper_bound - lower_bound)

## exponential averages

In [ ]:
def ema_bollinger_strategy(price, short_ema=30, long_ema=200, bb_window=50, bb_std=3, vol_window=252):

    # Exponential Moving Averages
    ema_short = price.ewm(span=short_ema, adjust=False).mean()
    ema_long = price.ewm(span=long_ema, adjust=False).mean()

    # Bollinger Bands
    sma = price.rolling(bb_window).mean()
    std = price.rolling(bb_window).std()
    upper_band = sma + bb_std * std
    lower_band = sma - bb_std * std

    threshold = 0.05

    # Trend Signal (EMA Crossover)
    trend_signal = np.where((ema_short > ema_long * (1 + threshold)), 1,
                          np.where((ema_short < ema_long * (1 - threshold)), -1, 0))

    # Bollinger Band Signal (Mean Reversion)
    bb_signal = np.where(price > upper_band, -1,
                        np.where(price < lower_band, 1, 0))

    # Combined Signal (sum then clip between -2 and 2)?
    position = trend_signal + bb_signal
    position =  position = soft_clip(position, lower_bound=-1, upper_bound=1, smoothness=10)
    

    # Scale by volatility (dynamic position sizing)
    # vol = price.pct_change().rolling(vol_window).std()

   # Calculate portfolio returns (mean return of all instruments)
    portfolio_returns = price.pct_change().mean(axis=1)

    # Calculate the rolling volatility of the portfolio (overall portfolio volatility)
    vol = portfolio_returns.rolling(vol_window).std()

    # Scale by overall portfolio volatility (dynamic position sizing)
    # Broadcast 'vol' to match shape of 'position' (5443, 27)
    vol = vol.values[:, None]  # Make 'vol' a column vector (5443, 1)
    
    # Position scaling using portfolio volatility
    position = position / vol
    
    # Convert to DataFrame with same index as input
    position = pd.DataFrame(position, index=prices.index, columns=prices.columns)
    
    return position


position = ema_bollinger_strategy(prices)

In [ ]:
evaluation.plot_key_figures(position, prices)

## save your positions to a file

In [ ]:
position.to_csv('results.csv')